In [1]:
# imports
import os
import time
import pandas as pd
from skimage.measure import label,regionprops
from skimage.morphology import binary_opening, erosion, dilation, ball
from skimage.segmentation import watershed
from scipy import ndimage
from sklearn.mixture import BayesianGaussianMixture
from skimage.morphology import opening, closing, ball
from skimage.filters import gaussian
from skimage.transform import rescale
from scipy.stats import entropy
import numpy as np
import nrrd
from skimage.segmentation import morphological_geodesic_active_contour as gac
from skimage.segmentation import inverse_gaussian_gradient as igg
from scipy.ndimage.morphology import binary_fill_holes
from scipy.stats import entropy
# from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

In [2]:
''' 
SUPPLEMENTARY FUNCTION : createSubVolumes
This function isolates the sub-volume of the image that we are interested in. 
This way, we can perform operations only on the pixels containing lesion and surrounding pixels (executes faster).
    
    INPUT:
            image         - the original CT volume;
            image_dict    - dictionary with CT metadata;
            mask          - the original mask with radiologist-defined ROIs;
            lprop         - regionprops object for the user-defined start point.
            
    OUTPUT:
            cylinder_subV - cylinder mask, which occupies same space as both image and mask sub-volumes;
            image_subV    - image sub-volume, which contains lesion plus surrounding voxels;
            mask_subV     - ground truth labels, which occupies same space as both image and cylinder sub-volumes;
            R             - cylinder radius, for downstream calculations.
            
'''

def createSubVolumes(image,image_dict,mask,lprop):
    
    # construct cylinder_image
    bbox = lprop.bbox  
    bbox_dims = [bbox[3]-bbox[0],bbox[4]-bbox[1],bbox[-1]-bbox[2]]
    
    # take a circle with radius larger than bounding box
#   R = bbox_dims[0]/2 + bbox_dims[1]/2  
    R = max(bbox_dims)*5/8       
    
    # isolate the centroid of the nodule -- this will be our preliminary marker 
    j,i,k = [round(i) for i in lprop.centroid] 

    circle_mask = np.zeros((image.shape[0],image.shape[1]),dtype=bool)
    num_rows,num_cols = circle_mask.shape

    row,col = np.meshgrid(range(num_rows),range(num_cols))
    circle_mask[((row-i)**2+(col-j)**2)<R**2] = True
            
    # now we have the initial contour (that will theoretically be user-defined for the clinical dataset)
    # next step: extend the circle in both superior and inferior directions to obtain a cylinder

    # determine the number of slices required in the z direction for full coverage
    num_slices = int(np.ceil((float(image_dict['pixel_spacing']) * R) / float(image_dict['slice_thickness'])/2))

    cylinder_image = np.zeros((image.shape[0],image.shape[1],image.shape[2]),dtype=bool)

    for i in range(k-num_slices,k+num_slices+1):
        cylinder_image[:,:,i] = circle_mask
                
    # label the mask into connected regions
    mask_labels = label(cylinder_image)
    mask_props = regionprops(mask_labels)

    coords = mask_props[0].bbox

    cylinder_subV = mask_props[0].image
    image_subV = image[coords[0]:coords[3],coords[1]:coords[4],coords[2]:coords[-1]]
    mask_subV = mask[coords[0]:coords[3],coords[1]:coords[4],coords[2]:coords[-1]]
    
    return cylinder_subV,image_subV,mask_subV,R

In [3]:
''' 
SUPPLEMENTARY FUNCTION : createSubVolumesClinical
This function isolates the sub-volume of the image that we are interested in. 
This way, we can perform operations only on the pixels containing lesion and surrounding pixels (executes faster).
    
    INPUT:
            image         - the original CT volume;
            image_dict    - dictionary with CT metadata;
            mask          - the original mask with radiologist-defined ROIs;
            lprop         - regionprops object for the user-defined start point.
            
    OUTPUT:
            image_subV    - image sub-volume, which contains lesion plus surrounding voxels;
            mask_subV     - ground truth labels, which occupies same space as both image and cylinder sub-volumes.
            
'''

def createSubVolumesClinical(image,mask,lprop):
    
    # get the bounding box for the user-defined mask in image coordinates
    coords = lprop.bbox 
    row = 3  # take an extra 3 voxels on either side of bounding box for row and col
    slc = 1  # take an extra voxel on either side of bounding box for axial slices
    
    img = image.copy()
    msk = mask.copy()
    
    # crop the image and mask to the expanded bounding box
    image_subV = img[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    mask_subV = msk[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    
    return image_subV,mask_subV

In [4]:
''' 
SUPPLEMENTARY FUNCTION : determineThreshold
This function determines a threshold separating candidate lesion voxels from surrounding lung paremchyma. 
It leverages the BayesianGaussianMixture functionality from sklearn. 
Since the pre-defined sub-volume contains both lesion and lung parenchyma pixels, density distribution 
of the pixels could be modeled by two Gaussian distributions: P(x|lesion) and P(x|parenchyma), 
where x was the pixel density. The mean values and variations of the two Gaussian distributions were 
then estimated by the expectation-maximization method.
    
    INPUT:
            image_subV - the CT sub-volume;
            mask_subV  - mask occupying the same space as the image_subV.
            
    OUTPUT:
            threshold     - floating point threshold (anything above this threshold is a candidate lesion voxel)
                     
'''

def determineThreshold(image_subV,mask_subV):
    
    # isolate the intensities within the mask
    intensities = image_subV[mask_subV]

    hist, bin_edges = np.histogram(intensities, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])

    classif = BayesianGaussianMixture(n_components=2)
    classif.fit(intensities.reshape(-1,1))

    return np.mean(classif.means_)

In [5]:
''' 
SUPPLEMENTARY FUNCTION : resampleVolumes
This function resamples the sub-volume and corresponding mask to isotropic voxel size (1mm, 1mm, 1mm). 
    
    INPUT:
            image     - the CT sub-volume;
            mask      - the mask occupying the same space as the image_subV;
            meta      - dictionary with original image and/or mask metadata (they are the same so either dictionary works).
            
    OUTPUT:
            img_RS    - the resampled CT sub-volume;
            msk_RS    - the resampled mask occupying the same space as the resampled CT sub-volume.
                     
'''

def resampleVolumes(image,mask,meta):

    dims = [meta['space directions'][0][0],meta['space directions'][0][0],meta['space directions'][-1][-1]]

    scaleFac = []
            
    for d in dims:
        if d < 1:
            scaleFac.append(1/float(d))
        else:
            scaleFac.append(float(d))
 
    # note the logical expression for the mask b/c the output from rescale is not binary
    msk_RS = rescale(mask, scale = (scaleFac[0],scaleFac[1],scaleFac[2]), preserve_range = True, anti_aliasing=None) > 0.5
    img_RS = rescale(image, scale = (scaleFac[0],scaleFac[1],scaleFac[2]), preserve_range = True, anti_aliasing=True)
            
    return img_RS,msk_RS

In [6]:
''' 
SUPPLEMENTARY FUNCTION : calcStats
This function isolates sub-regions of the segmented lesion (core, periphery, etc.) and calculates statistics of interest. 

    INPUT:
            image     - the resampled CT sub-volume;
            mask      - the resampled mask occupying the same space as the image_subV;
            r         - optional argument (specified to 2mm, which is appropriate for the resampled sub-volume).
            
    OUTPUT:
            stats     - statistics of interest for the sub-regions of interest (mean intensity and entropy).
                     
'''

def prepArray(arr):
    
    # get the range of arr
    r = np.ptp(arr)
    hist, bin_edges = np.histogram(arr,bins=10,density=True)
    
    return hist


def calcStats(image,mask,r=2):
    
    # put the masks into a list
    all_masks = []
    
    # stats 
    stats = []
    
    # (0) lesion
    all_masks.append(mask)
    
    # (1) lesion core
    all_masks.append(erosion(mask,ball(radius = r)))
    
    # (2) lesion interior rim
    all_masks.append(np.logical_and(mask,~erosion(mask,ball(radius = r))))
    
    # (3) lesion exterior rim
    all_masks.append(np.logical_and(~mask,dilation(mask,ball(radius = r))))
    
    # iterate through the list of masks to calculate the stats
    # CODE HERE
    for i in range(len(all_masks)):
        
        if ~all_masks[i].any():
            stats.append(np.nan)
            stats.append(np.nan)
        else:
            stats.append(np.mean(image[all_masks[i]]))
            stats.append(entropy(prepArray(image[all_masks[i]])))
            
    if np.nanmean(stats[0:-1:2]) > 0:
  
        for i in range(len(stats)):
            if i%2 == 1:
                continue
            else:
                stats[i] = stats[i] - 1024
        
        
    return stats

In [7]:
# lists of image/mask files
img_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images'
msk_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Masks'

all_images = sorted([os.path.join(img_path,f) for f in os.listdir(img_path) if 'nrrd' in f])
all_masks = sorted([os.path.join(msk_path,f) for f in os.listdir(msk_path) if 'nrrd' in f])

<div class="alert alert-block alert-info">
<b>NOTE:</b>
    
The `all_images` and `all_masks` variables contain pointers to individual files. The files in these folders are not separated by time point. Take the first four pointers in the `all_images` variable as an example:
<br>   
`./Images/112001_BL_image.nrrd`,<br>
`./Images/112001_C2_image.nrrd`,<br>
`./Images/112002_BL_image.nrrd`,<br>
`./Images/112002_C2_image.nrrd`,...<br>
<br>   
The first six digits represent a unique subject identifier, and the two letters that follow encode the timepoint, where **BL** is the baseline or first measurement and **C2** is the measurement after 2 cycles of chemotherapy or second measurement.
</div>

In [8]:
# first pass of segmentation pipeline
def segPipeline(imgList,mskList):
    
    # import the trial arm key -- the arm that received the hypoxia drug is encoded as 1
    key = pd.read_csv('./trial_arm_key.csv')
    stats_columns = ['lesion mu','lesion S','core mu','core S','interior mu','interior S','exterior mu','exterior S']
    
    # initialize lists for segmentation accuracy
    usubjid = []
    timepoint = []
    trial_arm = []
    lesion_index = []
    lesion_volume = []
    stats = []
    error_log = []
    algo_skip = []
    
    for i in range(len(imgList)):
        
        # read image and corresponding mask
        img_V,img_d = nrrd.read(imgList[i])
        msk_V,msk_d = nrrd.read(mskList[i])
        
        # calculate image and mask voxel volumes
        patient_id,time_id,fileID = os.path.basename(imgList[i]).split('_')
        msk_voxel_volume = msk_d['space directions'][0][0]**2 * msk_d['space directions'][-1][-1] # in mm^3
        img_voxel_volume = img_d['space directions'][0][0]**2 * img_d['space directions'][-1][-1] # in mm^3
        
#         print('mask voxel volume: ',msk_voxel_volume)
#         print('image voxel volume: ',img_voxel_volume)
        
        # if the image and mask do not have the same voxel volume, do not process
        if abs(msk_voxel_volume - img_voxel_volume) > 0.0001:
            print('ERROR, {}: image and mask must have same voxel volumes'.format(imgList[i]))
            error_log.append(imgList[i])
            continue
            
        # label the mask into connected regions
        lesion_labels,num_lesions = label(msk_V,return_num=True)
        lesion_props = regionprops(lesion_labels)

        # track lesion number
        lesion_count = 0
        
        # for every lesion in the mask
        for obj in lesion_props:
            
            usubjid.append(patient_id)
            timepoint.append(time_id)
            trial_arm.append(key.loc[key['USUBJID'] == int(patient_id),'ARM'].item())
            lesion_index.append(lesion_count)
            lesion_count += 1
            
            # create the sub-volume of interest within the CT volume
            img_subV,msk_subV = createSubVolumesClinical(img_V,msk_V,obj)
            msk_subV = msk_subV > 0
            
            # calculate threshold using EM
            threshold = determineThreshold(img_subV,msk_subV)
            
            # label the cylinder sub-volume into connected regions
            msk_subV_labels = label(msk_subV)
            msk_subV_props = regionprops(msk_subV_labels)
            
            # get the dimensions of the user-defined mask
            bbox = msk_subV_props[0].bbox  
            bbox_dims = [bbox[3]-bbox[0],bbox[4]-bbox[1],bbox[-1]-bbox[2]]
            R = max(bbox_dims) / 2 

            # coordinates for the user-defined lesion centroid
            msk_j,msk_i,msk_k = [round(i) for i in msk_subV_props[0].centroid]

            binary_img = np.logical_and(np.logical_and(img_subV > threshold,img_subV > -850),img_subV < 200)
            binary_img[~msk_subV] = False
            binary_img_centroid = closing(opening(binary_img,ball(radius=int(0.1*R))),ball(radius=int(0.1*R)))
            
            # construct a marker image for the watershed
            marker_image = np.zeros((img_subV.shape[0],img_subV.shape[1],img_subV.shape[2]),dtype=np.uint8)
            marker_image[~binary_img] = 2         # background/lung parenchyma voxels
            marker_image[msk_j,msk_i,msk_k] = 1   # user-defined lesion centroid

            # denoise image sub-volume
            denoised = gaussian(img_subV,multichannel=False)

            # try some funky stuff
            max_image = dilation(denoised,ball(2))
            min_image = erosion(denoised,ball(2))

            gradient_image = max_image - min_image
            gradient_image[~msk_subV] = np.min(gradient_image)

            # create distance matrix for the 3D bowl function
            row,col,slc = np.meshgrid(range(gradient_image.shape[1]),range(gradient_image.shape[0]),range(gradient_image.shape[2]))
            dist_matrix = np.sqrt((round(msk_j) - row)**2 + (round(msk_i) - col)**2 + ((round(msk_k) - slc) * msk_d['space directions'][-1][-1])**2)
            dist_matrix[dist_matrix>=R] = R
            dist_matrix = dist_matrix / R
            
            # modify the gradient image for watershed
            mod_gradient = gradient_image * dist_matrix
            
            # perform watershed segmentation
            water_initial = ~watershed(mod_gradient, marker_image,connectivity=2)
            
            # label the watershed mask into connected regions
            water_labels = label(water_initial,background = np.min(water_initial))
            water_props = regionprops(water_labels)
            
            # find the largest region
            water_areas = [water_props[i].area for i in range(len(water_props))]
            ind = np.where(water_areas == np.max(water_areas))[0][0]

            # create the mask
            water_mask = water_labels == water_props[ind].label
            
            # FINALLY this initial segmentation is fed into the active contours function for further refinement
#             refined_mask = gac(igg(img_subV), iterations = 3, init_level_set=water_mask, threshold = 0.5)
            
#             refined_mask = gac(igg(img_subV), iterations = 15, init_level_set=binary_img, threshold = 0.5,balloon = -2)
            
            # fill any holes in the final mask result (unlikely, but you never know)
#             refined_mask = binary_fill_holes(refined_mask)

            v = water_mask.sum() * msk_voxel_volume
    
            # if the volume is less than 25% of what the user defined, the algorithm clearly failed :(
            if v < (msk_subV.sum() * msk_voxel_volume) / 4:
                img_RS,msk_RS = resampleVolumes(img_subV,msk_subV,img_d)
                # take the core of the user-defined volume
                user_mask = erosion(msk_RS,ball(radius=2))
                v = user_mask.sum()
                lesion_volume.append(v)
                stats.append(calcStats(img_RS,user_mask))
                algo_skip.append(imgList[i])
                
            else:
                lesion_volume.append(v)
                img_RS,msk_RS = resampleVolumes(img_subV,water_mask,img_d)
                stats.append(calcStats(img_RS,msk_RS))
            
            print('Lesion volume: {:.2f} mm^3'.format(v))
            
            
#             extra_stats = AEFunction(img_subV,water_mask)  # could be refined_mask

            output_dict = { 'USUBJID' : usubjid,         # patient identifier (6-digit number)
                            'TIMEPT'  : timepoint,       # timepoint identifier (2-letter code -- BL baseline, C2 cycle 2)
                            'ARM'     : trial_arm,       # trial arm (0 control group, 1 experimental group)
                            'LSNIND'  : lesion_index,    # lesion index (for patients with more than 1 lesion)
                            'LSNVOL'  : lesion_volume}   # lesion volume (in mm^3)
    
        print('Processed image {}/{}: {}% complete.'.format(i+1,len(imgList),float(i+1)/len(imgList)*100))
        # concatenate the volume and stats information
        frames = [pd.DataFrame(output_dict),pd.DataFrame(stats,columns=stats_columns)] 
    
    return pd.concat(frames,axis=1),error_log,algo_skip,img_subV
        

# run the pipeline
start_time = time.time()
df,error_log,algo_skip,test_img = segPipeline(all_images,all_masks)
print('Total processing time: {:.2f} minutes.'.format((time.time()-start_time)/60))

Lesion volume: 3108.96 mm^3
Processed image 1/84: 1.1904761904761905% complete.
Lesion volume: 10918.00 mm^3
Processed image 2/84: 2.380952380952381% complete.
Lesion volume: 2523.57 mm^3
Lesion volume: 4649.56 mm^3
Lesion volume: 1278.00 mm^3
Processed image 3/84: 3.571428571428571% complete.
Lesion volume: 3786.07 mm^3
Lesion volume: 6308.52 mm^3
Lesion volume: 531.00 mm^3
Processed image 4/84: 4.761904761904762% complete.
Lesion volume: 2398.00 mm^3
Lesion volume: 629.30 mm^3
Lesion volume: 2763.00 mm^3
Lesion volume: 632.00 mm^3
Lesion volume: 4679.00 mm^3
Lesion volume: 4948.00 mm^3
Lesion volume: 675.00 mm^3
Lesion volume: 607.03 mm^3
Lesion volume: 1168.00 mm^3
Processed image 5/84: 5.952380952380952% complete.
Lesion volume: 968.00 mm^3
Lesion volume: 5210.00 mm^3
Lesion volume: 2943.00 mm^3
Lesion volume: 640.00 mm^3
Lesion volume: 2681.00 mm^3
Lesion volume: 2225.00 mm^3


/Users/EL-CAPITAN-2016/anaconda2/envs/spyder/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


Lesion volume: 644.53 mm^3
Lesion volume: 4942.00 mm^3
Lesion volume: 672.00 mm^3
Processed image 6/84: 7.142857142857142% complete.
Lesion volume: 128.00 mm^3
Processed image 7/84: 8.333333333333332% complete.
Lesion volume: 304.00 mm^3
Processed image 8/84: 9.523809523809524% complete.
Lesion volume: 2672.00 mm^3
Lesion volume: 318.00 mm^3
Processed image 9/84: 10.714285714285714% complete.
Lesion volume: 1115.00 mm^3
Lesion volume: 114.00 mm^3
Processed image 10/84: 11.904761904761903% complete.
Lesion volume: 5102.00 mm^3
Lesion volume: 497.00 mm^3
Processed image 11/84: 13.095238095238097% complete.
Lesion volume: 2070.00 mm^3
Lesion volume: 344.00 mm^3
Processed image 12/84: 14.285714285714285% complete.
Lesion volume: 630.00 mm^3
Processed image 13/84: 15.476190476190476% complete.
Lesion volume: 820.00 mm^3
Processed image 14/84: 16.666666666666664% complete.
Lesion volume: 120.00 mm^3
Processed image 15/84: 17.857142857142858% complete.
Lesion volume: 90.00 mm^3
Processed imag

/Users/EL-CAPITAN-2016/anaconda2/envs/spyder/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


Lesion volume: 1864.00 mm^3
Lesion volume: 2089.00 mm^3
Processed image 32/84: 38.095238095238095% complete.
Lesion volume: 36172.46 mm^3
Lesion volume: 24877.66 mm^3
Processed image 33/84: 39.285714285714285% complete.
Lesion volume: 96732.00 mm^3
Lesion volume: 101313.00 mm^3
Processed image 34/84: 40.476190476190474% complete.
Lesion volume: 1269.00 mm^3
Lesion volume: 22676.00 mm^3
Lesion volume: 795.00 mm^3
Processed image 35/84: 41.66666666666667% complete.
Lesion volume: 169602.00 mm^3
Lesion volume: 9530.00 mm^3
Lesion volume: 6672.00 mm^3
Processed image 36/84: 42.857142857142854% complete.
Lesion volume: 1516.00 mm^3
Lesion volume: 534.00 mm^3
Lesion volume: 1680.00 mm^3
Lesion volume: 1654.00 mm^3
Lesion volume: 969.00 mm^3


...

Lesion volume: 1975.00 mm^3
Lesion volume: 4717.00 mm^3
Processed image 73/84: 86.90476190476191% complete.
ERROR, /Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/321014_C2_image.nrrd: image and mask must have same voxel volumes
Lesion volume: 1494.00 mm^3
Lesion volume: 723.00 mm^3
Processed image 75/84: 89.28571428571429% complete.
Lesion volume: 668.00 mm^3
Lesion volume: 499.00 mm^3
Processed image 76/84: 90.47619047619048% complete.
Lesion volume: 13674.00 mm^3
Lesion volume: 3005.00 mm^3
Processed image 77/84: 91.66666666666666% complete.
Lesion volume: 1147.09 mm^3
Lesion volume: 404.00 mm^3
Processed image 78/84: 92.85714285714286% complete.
ERROR, /Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/321018_BL_image.nrrd: image and mask must have same voxel volumes
Lesion volume: 1028.00 mm^3
Lesion volume: 1618.00 mm^3
Processed image 80/84: 95.23809523809523% complete.


/var/folders/x7/r3l4_w3n3t1dlrytrq4t48180000gn/T/ipykernel_95373/573609983.py:55: RuntimeWarning: Mean of empty slice
  if np.nanmean(stats[0:-1:2]) > 0:


Lesion volume: 0.00 mm^3


/var/folders/x7/r3l4_w3n3t1dlrytrq4t48180000gn/T/ipykernel_95373/573609983.py:55: RuntimeWarning: Mean of empty slice
  if np.nanmean(stats[0:-1:2]) > 0:


Lesion volume: 0.00 mm^3
Lesion volume: 7.00 mm^3
Lesion volume: 908.00 mm^3
Lesion volume: 335.00 mm^3
Processed image 81/84: 96.42857142857143% complete.
Lesion volume: 1844.00 mm^3
Lesion volume: 1297.00 mm^3
Lesion volume: 1147.00 mm^3
Lesion volume: 5029.00 mm^3
Lesion volume: 254.00 mm^3
Processed image 82/84: 97.61904761904762% complete.
Lesion volume: 458.00 mm^3
Lesion volume: 2202.00 mm^3
Lesion volume: 458.00 mm^3
Processed image 83/84: 98.80952380952381% complete.
Lesion volume: 265.00 mm^3
Lesion volume: 1867.00 mm^3
Lesion volume: 71.00 mm^3
Processed image 84/84: 100.0% complete.
Total processing time: 9.27 minutes.


In [23]:
df[:26]

,USUBJID,TIMEPT,ARM,LSNIND,LSNVOL,lesion mu,lesion S,core mu,core S,interior mu,interior S,exterior mu,exterior S
0,112001,BL,0,0,3108.962354,-135.368490,1.876100,-14.042021,1.281733,-252.636750,2.139033,-622.675571,2.047433
1,112001,C2,0,0,10918.000000,-234.217878,2.065057,-119.786704,1.893842,-397.298822,2.152227,-517.993819,1.985604
2,112002,BL,1,0,2523.569095,-84.723585,1.709494,12.142250,1.333188,-189.616346,2.136189,-456.124858,2.211591
3,112002,BL,1,1,4649.557022,-5.373325,1.358220,41.269862,1.494562,-82.999330,2.003649,-310.904060,2.118805
4,112002,BL,1,2,1278.000000,-442.226350,2.258485,-236.747292,1.990409,-518.207352,2.233228,-731.605519,1.938434
5,112002,C2,1,0,3786.067651,-4.476244,1.781860,50.436577,1.571440,-57.312531,2.001463,-305.583941,2.127301
6,112002,C2,1,1,6308.523303,-6.608159,1.516177,11.193116,1.893799,-28.498916,1.920511,-202.706330,1.955823
7,112002,C2,1,2,531.000000,-210.094915,2.063398,-61.285714,2.043650,-230.126923,2.086429,-516.209211,2.122099
8,113007,BL,1,0,2398.000000,-284.369670,2.261511,-31.083130,1.815515,-327.077829,2.276176,-600.286285,1.968337
9,113007,BL,1,1,629.296875,-26.329107,1.863594,81.383292,1.704076,-96.747825,2.105589,-432.891100,2.120532


In [24]:
error_log

['/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/191026_BL_image.nrrd',
 '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/194020_BL_image.nrrd',
 '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/321014_C2_image.nrrd',
 '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/321018_BL_image.nrrd']

In [11]:
len(algo_skip)

183

In [12]:
# testing

lst = [2,0,3,0,4,0,5,0]
if np.mean(lst[0:-1:2]) > 0:
#     lst[::2] = lst[::2] - 1024
    
    for i in range(len(lst)):
        if i%2 == 1:
            continue
        else:
            lst[i] = lst[i] - 1024
    
lst

[-1022, 0, -1021, 0, -1020, 0, -1019, 0]

In [13]:
np.nan-1024


nan

In [14]:
msk_V,msk_d = nrrd.read(all_masks[2])
msk_d

OrderedDict([('type', 'unsigned char'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([512, 512, 161])),
             ('space directions',
              array([[0.77148, 0.     , 0.     ],
                     [0.     , 0.77148, 0.     ],
                     [0.     , 0.     , 4.     ]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('encoding', 'gzip'),
             ('space origin', array([0., 0., 0.])),
             ('Segment0_Color', '0.501961 0.682353 0.501961'),
             ('Segment0_ColorAutoGenerated', '1'),
             ('Segment0_Extent', '135 412 252 357 136 154'),
             ('Segment0_ID', 'Segment_1'),
             ('Segment0_LabelValue', '1'),
             ('Segment0_Layer', '0'),
             ('Segment0_Name', 'TL01_BL'),
             ('Segment0_NameAutoGenerated', '0'),
             ('Segment0_Tags',
              'Segmentation.Status:inprogress|TerminologyEntry:Segmentation c

In [15]:
props = regionprops(label(msk_V))
props[0].bbox

(135, 285, 149, 163, 312, 155)

In [16]:
df

,USUBJID,TIMEPT,ARM,LSNIND,LSNVOL,lesion mu,lesion S,core mu,core S,interior mu,interior S,exterior mu,exterior S
0,112001,BL,0,0,3108.962354,-135.368490,1.876100,-14.042021,1.281733,-252.636750,2.139033,-622.675571,2.047433
1,112001,C2,0,0,10918.000000,-234.217878,2.065057,-119.786704,1.893842,-397.298822,2.152227,-517.993819,1.985604
2,112002,BL,1,0,2523.569095,-84.723585,1.709494,12.142250,1.333188,-189.616346,2.136189,-456.124858,2.211591
3,112002,BL,1,1,4649.557022,-5.373325,1.358220,41.269862,1.494562,-82.999330,2.003649,-310.904060,2.118805
4,112002,BL,1,2,1278.000000,-442.226350,2.258485,-236.747292,1.990409,-518.207352,2.233228,-731.605519,1.938434
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,321023,BL,1,1,2202.000000,-42.112540,1.422978,47.566232,0.523282,-102.261277,1.827964,-429.177917,2.257094
198,321023,BL,1,2,458.000000,-212.459373,2.079825,35.001356,1.780208,-234.207703,2.141160,-623.912315,1.959648
199,321023,C2,1,0,265.000000,-201.034801,2.078595,NaN,NaN,-201.034801,2.078595,-484.758540,2.194693
200,321023,C2,1,1,1867.000000,-25.850942,1.623093,55.624095,1.283712,-78.278980,1.846779,-480.009377,2.159126


In [17]:
msk_V,msk_d = nrrd.read('/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/112002_C2_mask.nrrd')
img_V,img_d = nrrd.read('/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/112002_BL_image.nrrd')

In [18]:
msk_d

OrderedDict([('type', 'short'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([512, 512, 368])),
             ('space directions',
              array([[0.97656, 0.     , 0.     ],
                     [0.     , 0.97656, 0.     ],
                     [0.     , 0.     , 5.     ]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'raw'),
             ('space origin', array([0., 0., 0.]))])

In [19]:
img_d

OrderedDict([('type', 'short'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([512, 512, 161])),
             ('space directions',
              array([[0.77148, 0.     , 0.     ],
                     [0.     , 0.77148, 0.     ],
                     [0.     , 0.     , 4.     ]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('space origin', array([0., 0., 0.]))])

In [20]:
props = regionprops(label(msk_V))
props

In [21]:
df.to_csv('results_5Apr2022.csv')